<a href="https://colab.research.google.com/github/maxelOA/formula-1-data/blob/main/erpy2set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Paqueterias**

In [1]:
import requests as r
import json 
import pandas as pd
import numpy as np
import pdb

# **UNRAVEL PACK**


In [2]:
########################################
######### Primer dic en Df listas ###### v1.1
########################################

def first_dict(column):

  column=column[contains_dict(column)]
  
  return column.iloc[0]

########################################
######### Existe en ambas listas ####### v1.1
########################################

def it_contains(list1,list2):

  if any((match := item) in list1 for item in list2):
    return True

  else:
    return False

########################################
######### Restructura Dataframe ######## V1.1
########################################

def dataframe_res(Df1,location,Df2):

    Df2_names=Df2.columns.tolist()

    for i in Df2.columns:
      Df1.insert(loc=location, column=i,value=Df2[i])
      location+=1
    
    return Df1

########################################
###### Checa si el objeto es Dict ###### V1.0
########################################

def is_dict(mydic):
  if type(mydic)==dict:
    return True
  else:
    return False

########################################
#### Checa si la columna tiene dict ####  V1.0
########################################

def contains_dict(interest_column):

  boolean_list=[]

  for i in range(len(interest_column)):

    if is_dict(interest_column[i]):
      boolean_list.append(True)
    else:
      boolean_list.append(False)

  return boolean_list

########################################
######### Dict a Columnas ############## V1.0
########################################

def expand(chosen_column, join_id_column,desired_column_name,prefix,f_dict):

    dictionary = []

    len_dict=len(f_dict)
    empty_dict=dict(zip(f_dict.keys(), [np.nan]*len_dict))

    #len_dict=len(chosen_column[0])
    #empty_dict=dict(zip( chosen_column[0].keys(), [np.nan]*len_dict))

    for i,k in zip(chosen_column,join_id_column):

        if pd.isnull(i)==True:
            i=empty_dict
        if i:
           # i = i.replace("'", '"').replace(" ", "")
           # i = json.loads(i)
            dict1 = {desired_column_name: k }
            new_dict = {**dict1, **i}
            dictionary.append(new_dict)

            Df=pd.DataFrame(dictionary)
            Df_names=Df.columns.tolist()
            Df_names.remove(desired_column_name)

    for i in Df_names:  
      Df.rename(columns={i:prefix+'_'+i}, inplace=True)

    return Df

########################################
######### Dict a Columnas nokey ######## V1.0
########################################

def expand_noKey(chosen_column, prefix,f_dict):

    dictionary = []

    len_dict=len(f_dict)
    empty_dict=dict(zip(f_dict.keys(), [np.nan]*len_dict))

    #len_dict=len(chosen_column[0])
    #empty_dict=dict(zip( chosen_column[0].keys(), [np.nan]*len_dict))
   

    for i in chosen_column:

        if pd.isnull(i)==True:
            i=empty_dict
        if i:
           # i = i.replace("'", '"').replace(" ", "")
           # i = json.loads(i)  
            new_dict = {**i}
            dictionary.append(new_dict)

            Df=pd.DataFrame(dictionary)
            Df_names=Df.columns.tolist()

    for i in Df_names:  
      Df.rename(columns={i:prefix+'_'+i}, inplace=True)

    return Df

########################################
## Regresa las columnas estructuradas ##  V1.0
########################################

def st_df(Df):

 structure_columns_names=[]
 columns_names= Df.columns.tolist()

 for i in columns_names:

    if True in contains_dict(Df[i]):
      structure_columns_names.append(i)
  
 return structure_columns_names

########################################
######### Todas las columnas ########### v1.1
########################################

def unravel(Df, join_id_column,desired_column_name): 
  
  expand_names=st_df(Df)

  if it_contains(expand_names,Df.columns.tolist())==True:
       for i in expand_names:


         df_aux=expand(Df[i].tolist(), join_id_column.tolist(), desired_column_name,i)
         new_cols=df_aux.loc[:, df_aux.columns!=desired_column_name]      

         position_insert=Df.columns.get_loc(i)
         Df=dataframe_res(Df,position_insert,new_cols)
         Df.drop(i, inplace=True, axis=1)

         unravel(Df,join_id_column,desired_column_name)

         return Df
  else:
         return Df


########################################
####### Todas las columnas nokey ####### v1.1
########################################

def unravel_noKey(Df):
  expand_names=st_df(Df)

  if it_contains(expand_names,Df.columns.tolist())==True:
       for i in expand_names:


         df_aux=expand_noKey(Df[i].tolist(),i,first_dict(Df[i]))

         position_insert=Df.columns.get_loc(i)
         Df=dataframe_res(Df,position_insert,df_aux)
         Df.drop(i, inplace=True, axis=1)

         unravel_noKey(Df)

         return Df
  else:
         return Df

# **QUERY PACK**



In [18]:
########################################
##### recupera la tabla de laps  ####### v1.1
########################################


def laps_tbl(year):

  data=pd.DataFrame()
  n=num_races(year)

  for j in list(range(1,n+1)):
    k=max_laps(year,j)
    
    for i in list(range(1,k+1)):
      url=f'http://ergast.com/api/f1/{year}/{j}/laps/{i}.json'

      aux_data=pd.DataFrame(r.get(url).json()['MRData']['RaceTable']['Races'][0]['Laps'][0]['Timings'])
      aux_data[['lap','raceId']]=[i,j]
      data=data.append(aux_data)
    
  return data

########################################
#### recupera la tabla de pitstops  #### v1.1
########################################

def pitstops_tbl(year):

  data=pd.DataFrame()
  n=num_races(year)

  for i in list(range(1,n+1)):
    url=f'http://ergast.com/api/f1/{year}/{i}/pitstops.json'

    aux_data=pd.DataFrame(r.get(url).json()['MRData']['RaceTable']['Races'][0]['PitStops'])
    aux_data['raceId']=i
    data=data.append(aux_data)

  return data

########################################
# recupera la tabla de constructorResults  v1.1
########################################

def constructorResults_tbl(year):

  data=pd.DataFrame()
  n=num_races(year)

  for i in list(range(1,n+1)):

      url=f'https://ergast.com/api/f1/{year}/{i}/results.json'

      aux_data=pd.DataFrame(r.get(url).json()['MRData']['RaceTable']['Races'][0]['Results'])
      aux_data=aux_data[['Constructor','points','status']]
      unravel_noKey(aux_data)

      aux_data['raceId']=i
      data=data.append(aux_data)

  return data

########################################
#calcula en numero de vultas por carrera  v1.1
########################################

def max_laps(year,number_race):

  url=f'https://ergast.com/api/f1/{year}/{number_race}/results.json'
  data=r.get(url).json()
  rslts=pd.DataFrame(data['MRData']['RaceTable']['Races'][0]['Results'])['laps'].astype(int)
  
  return max(rslts)

############################################
#calcula en numero de carreras por temporada  v1.1
############################################

def num_races(year):
  races=pd.DataFrame()

  url=f'https://ergast.com/api/f1/{year}/races.json'
  data=r.get(url).json()
  races=pd.DataFrame(data['MRData']['RaceTable']['Races'])

  return len(races.axes[0])

############################################
# Regresa la cuerda con la primer letra Mayuscula  v1.1
############################################

def firstCap(string_toCap):
  cap=string_toCap[0].capitalize()

  adj_string=cap+string_toCap[1:]

  return adj_string

############################################
#    Ajusta el nombre de las tablas v1.1
############################################

def adj_name(tbl_name):

  if tbl_name == 'qualifyingResults':
    return 'qualifying'
  else:
    return tbl_name

############################################
#   Regrasa la tabla consultada Parent_child v1.1
############################################

def raw_table(table_name,year):
  Df=pd.DataFrame()
  
  dimension_tables={'seasons':'SeasonTable','drivers':'DriverTable','constructors':'ConstructorTable','circuits':'CircuitTable','status':'StatusTable','races':'RaceTable'}
  fact_tables={'driverStandings':['StandingsTable','StandingsLists'],'results':['RaceTable','Races'],'qualifyingResults':['RaceTable','Races'],'constructorStandings':['StandingsTable','StandingsLists']}

  depend_races=['pitstops','laps']
  depend_results=['constructorResults']

  total_races=num_races(year)
  tbl_names= list({**fact_tables,**dimension_tables})+depend_races+depend_results

  if table_name in tbl_names:
    
    
    if table_name in dimension_tables:

     url=f'https://ergast.com/api/f1/{year}/{adj_name(table_name)}.json'
     data=r.get(url).json() 

     locations=dimension_tables[table_name]
     Df=pd.DataFrame(data['MRData'][locations][firstCap(table_name)])

     unravel_noKey(Df)

    elif table_name in fact_tables:

      for i in list(range(1,total_races+1)):

           url=f'https://ergast.com/api/f1/{year}/{i}/{adj_name(table_name)}.json'
           data=r.get(url).json() 

           locations=fact_tables[table_name]
           aux_Df=pd.DataFrame(data['MRData'][locations[0]][locations[1]][0][firstCap(table_name)])
           unravel_noKey(aux_Df)
           aux_Df['racesId']=i
           aux_Df['year']=year
        
           Df=Df.append(aux_Df)

    elif table_name in depend_races:

      if table_name =='pitstops':
           Df=pitstops_tbl(year)

      else:
           Df=laps_tbl(year)
      
    else:
     Df=constructorResults_tbl(year)
   
  else:
     print("No table exist in the schema with that name")

  return Df

############################################
#  Regrasa la tabla consultada del esquema v1.1
############################################

def clean_table(raw_table,table_name):

  if table_name=='races':

    table=raw_table[['season','round','Circuit_circuitId','raceName','Circuit_circuitName','date','time','url','FirstPractice_date','SecondPractice_date','ThirdPractice_date','Sprint_date']]
    table.columns=['season','round','circuitId','raceName','circuitName','date','time','url','FirstPractice_date','SecondPractice_date','ThirdPractice_date','Sprint_date']

  elif table_name=='results': 

     table=raw_table[['raceId','year','Driver_driverId','Constructor_constructorId','number','grid','position','positionText','points','laps','Time_time','Time_millis','FastestLap_lap','FastestLap_rank','FastestLap_Time_time','FastestLap_AverageSpeed_speed','status']]
     table.columns=['raceId','season','driverId','constructorId','number','grid','position','positionText','points','laps','time','milliseconds','fastestLap','rank','fastestLapTime','fastestLapSpeed','status']

  elif table_name=='circuits':

     table=raw_table[['circuitId','circuitName','Location_locality','Location_country','Location_lat','Location_long','url']]
     table.columns=['circuitId','circuitName','locality','country','lat','long','url']

  elif table_name=='constructorStandings':

     table=raw_table[['raceId','Constructor_constructorId','points','position','positionText','wins']]
     table.columns=['raceId','constructorId','points','position','positionText','wins']
  
  elif table_name=='driverStandings':
     
     table=raw_table[['raceId','Driver_driverId','points','position','positionText','wins']]
     table.columns=['raceId','driverId','points','position','positionText','wins']

  elif table_name=='qualifyingResults':

     table=raw_table[['racesId','Driver_driverId','Constructor_constructorId','number','position','Q1','Q2','Q3']]
     table.columns=['raceId','driverId','constructorId','number','position','Q1','Q2','Q3']

  else:
    return raw_table

  return table

############################################
#  Regrasa la tabla consultada en un rango v1.1
############################################

  def query_range(table_name,initial_date,final_date):

  Df=pd.DataFrame()

  for i in list(range(initial_date,final_date+1)):

      aux_Df=raw_table(table_name,i)
      Df=Df.append(aux_Df)

  
  Df=clean_table(Df,table_name)

  return Df

In [8]:
from datetime import time

def adj_type(table):

  string_type=[]
  date_type=[]
  time_type=[]
  int_type=[]
  float_type=[]

In [27]:
############################################
#  Regrasa el esquema consultad en un rango v1.1
############################################

def full_schema(initial_date,final_date):
  tables=[]
  Df=pd.DataFrame()

  schema=['seasons','drivers','constructors','circuits','status','races','driverStandings','results','qualifyingResults','constructorStandings','pitstops','laps','constructorResults']

  for table in schema:

      Df=query_range(table,initial_date,final_date)
      tables.append(Df)

  return dict(zip(schema, tables))